In [1]:
!pip install -qq unsloth vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/

In [2]:
import re
import warnings

from unsloth import FastLanguageModel
from datasets import Dataset, load_dataset
from transformers import set_seed
from trl import GRPOConfig, GRPOTrainer
from vllm import SamplingParams

warnings.filterwarnings("ignore")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-02 15:29:03 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-02 15:29:03 [__init__.py:239] Automatically detected platform cuda.


Define hyperparameters

In [3]:
RANDOM_SEED = 42

MODEL_NAME_OR_PATH = "unsloth/Llama-3.2-3B-Instruct"
DATASET_NAME = "openai/gsm8k"

MAX_SEQ_LEN = 2048

set_seed(RANDOM_SEED)

Load backbone LLM for LoRA fine-tuning

In [4]:
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME_OR_PATH,
    max_seq_length=MAX_SEQ_LEN,
    load_in_4bit=False,
    fast_inference=True,
    max_lora_rank=8,
    gpu_memory_utilization=0.9,
)

==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 89.15%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.13 GB. Also swap space = 0 GB.
WARNING 06-02 15:29:16 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 06-02 15:29:36 [config.py:717] This model supports multiple tasks: {'reward', 'embed', 'generate', 'score', 'classi

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

...

INFO 06-02 15:35:28 [model_runner.py:1592] Graph capturing finished in 51 secs, took 0.24 GiB
INFO 06-02 15:35:28 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 63.05 seconds
Unsloth: Just some info: will skip parsing ['q_norm', 'k_norm', 'post_feedforward_layernorm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['q_norm', 'k_norm', 'post_feedforward_layernorm', 'pre_feedforward_layernorm']


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

In [5]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )

Apply LoRA to the LLM using the following LoRA setting:
- The layers applied LoRA are: q_proj, k_proj, v_proj, and o_proj.
- LoRA Rank: 8
- LoRA Alpla: 32

In [6]:
### TODO: Get LoRA applied to the LLM using the above hyperparameters
# HINT: use r, target_modules, and lora_alpha arguments
model = FastLanguageModel.get_peft_model(
    base_model,
    use_gradient_checkpointing="unsloth",
    random_state=RANDOM_SEED,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=32,
)
### END

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.5.9 patched 28 layers with 28 QKV layers, 28 O layers and 0 MLP layers.


Load the dataset from HuggingFace

In [7]:
dataset = load_dataset(DATASET_NAME, 'main', split="train")
dataset, dataset[0]

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

(Dataset({
     features: ['question', 'answer'],
     num_rows: 7473
 }),
 {'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
  'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'})

Clean the dataset

In [8]:
regex_answer_pattern = re.compile(r"(####)\s*(.*)",re.IGNORECASE)

extracted_samples = []
for item in dataset:
    question, raw_answer = item['question'].strip(), item["answer"].strip()
    answer_parts = raw_answer.split("####")
    answer = answer_parts[-1].strip()
    reasoning = "####".join(answer_parts[:-1]).strip()
    extracted_samples.append({"question": question, "answer": answer, "reasoning": reasoning})
print(extracted_samples[0])

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': '72', 'reasoning': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.'}


In [9]:
thinking_start, thinking_end = "<thinking>", "</thinking>"
answer_start, answer_end = "<answer>", "</answer>"

SYSTEM_PROMPT = f"""You are given a problem.
Think about the problem and provide your thought process.
Place it between {thinking_start} and {thinking_end}.
Then, provide your final answer between {answer_start} and {answer_end}"""

train_dataset = Dataset.from_list(extracted_samples)
train_dataset = train_dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",  "content": x["question"]},
    ],
    "answer": x["answer"],
})
train_dataset

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'reasoning', 'prompt'],
    num_rows: 7473
})

In [10]:
tokenized_dataset = train_dataset.map(
    lambda x: { "tokens": tokenizer.apply_chat_template(x["prompt"], add_generation_prompt=True, tokenize=True)},
    batched=True
)
max_len = max(tokenized_dataset.map(lambda x: {"length": len(x["tokens"])})["length"])

max_prompt_length = max_len + 1
print(f'max_prompt_length: {max_prompt_length}')

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

max_prompt_length: 285


Configure the training hyperparameters as follows:

- learning_rate=3e-5
- optim=paged_adamw_8bit
- per_device_train_batch_size=2
- gradient_accumulation_steps=4
- max_grad_norm=1
- num_train_epochs=1

In [11]:
training_args = GRPOConfig(
    learning_rate=3e-5,
    optim="paged_adamw_8bit",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_grad_norm=1,
    num_train_epochs=1,
    logging_first_step=True,
    max_steps=-1,
    save_steps=20,
    logging_steps=10,
    warmup_ratio=0.1,
    num_generations=2,
    max_prompt_length=max_prompt_length,
    max_completion_length=MAX_SEQ_LEN - max_prompt_length,
    report_to="none",
    output_dir="gsm8k_reasoning",
)

Define reward functions

In [12]:
match_format = re.compile(rf"^[\s]*{thinking_start}.+?{thinking_end}.*?{answer_start}(.+?){answer_end}[\s]*$", flags=re.MULTILINE | re.DOTALL)
match_numbers = re.compile(re.escape(answer_start) + r".*?(-?[\d\.,]{1,})", flags=re.MULTILINE | re.DOTALL)

def get_response_text(completion):
    return completion[0]["content"]

def match_format_exactly(completions, **kwargs):
    return [3.0 if match_format.search(get_response_text(c)) else 0.0 for c in completions]

def match_format_approximately(completions, **kwargs):
    def score_response(response):
        tags = [thinking_start, thinking_end, answer_start, answer_end]
        return sum(0.5 if response.count(tag) == 1 else -1.0 for tag in tags)

    return [score_response(get_response_text(c)) for c in completions]

def extract_with_regex(responses, pattern):
    return [m.group(1) if (m := pattern.search(r)) else None for r in responses]

def check_answer(prompts, completions, answer, **kwargs):
    responses = [get_response_text(c) for c in completions]
    extracted = extract_with_regex(responses, match_format)

    scores = []
    for guess, true_answer in zip(extracted, answer):
        if guess is None:
            scores.append(0.0)
        elif guess == true_answer:
            scores.append(3.0)
        elif guess.strip() == true_answer.strip():
            scores.append(1.5)
        else:
            scores.append(-1.5)
    return scores

def check_numbers(prompts, completions, answer, **kwargs):
    responses = [get_response_text(c) for c in completions]
    extracted = extract_with_regex(responses, match_numbers)

    scores = []
    for guess, true_answer in zip(extracted, answer):
        if guess is None:
            scores.append(0.0)
            continue
        try:
            guess_num = float(guess.strip().replace(",", ""))
            true_num = float(true_answer.strip())
            scores.append(1.5 if guess_num == true_num else -0.5)
        except Exception:
            scores.append(0.0)
    return scores

Train the LLM with GRPO trainer using the four defined reward functions: `match_format_exactly, match_format_approximately, check_answer, check_numbers`


In [13]:
set_seed(RANDOM_SEED)

### TODO: define the GRPOTrainer by filling out the arguments
grpo_trainer = GRPOTrainer(
    model=model,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    args=training_args,
    reward_funcs=[match_format_exactly, match_format_approximately, check_answer, check_numbers],
)
### END


grpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 1,869
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 4,587,520/3,217,337,344 (0.14% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.000000,0.125000,1.590990,258.000000,196.000000,292.000000,0.000000,258.000000,196.000000,292.000000,0.000000,0.000000,0.000000,-0.437500,1.590990,0.000000,0.000000,0.562500,0.776324
10,0.000000,0.722222,2.023111,206.513889,142.666667,295.666667,0.000000,206.513889,142.666667,295.666667,0.000006,0.375000,0.816463,-0.166667,1.288698,0.041667,0.479093,0.472222,0.852731


KeyboardInterrupt: 

In [ ]:
model.save_lora("gsm8k_reasoning")

Inference with `temperature=0.0` and maximum of 1024 tokens (`max_tokens=1024`)

In [ ]:
idx = RANDOM_SEED % len(train_dataset)
question, answer = train_dataset[idx]["question"], train_dataset[idx]["answer"]

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": question},
]

### TODO: fill in the code
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
output = model.fast_generate(
    [text],
    lora_request=model.load_lora("gsm8k_reasoning"),
    sampling_params=SamplingParams(
        temperature=0.0,
        max_tokens=1024
    ),
)
### END

output_text = output[0].outputs[0].text

print(f"Problem:", question)
print(f"Model Output:\n{output_text}")
print("Label:", answer)